### В ноутбуке: строим модель бинарной классификации

[GIT Link](https://github.com/newprolab/sber-spark-ds-10/blob/main/labs/lab03.md)

#### spark.sesion

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab03")
         .getOrCreate())

sc = spark.sparkContext

#### Задача
В вашем распоряжении имеется уже предобработанный и очищенный датасет с фактами покупок абонентами телепередач от компании E-Contenta. По доступным вам данным, нужно предсказать вероятность покупки других передач этими, а, возможно, и другими абонентами. При решении задачи запрещено использовать библиотеки pandas, sklearn (кроме sklearn.metrics), xgboost и другие. Если scikit-learn (например, но и другие тоже) обернут в классы Transformer и Estimator, то их можно использовать.


#### Описание данных
Для выполнения работы вам следует взять все файлы из папки на HDFS /labs/slaba03/.

In [3]:
!hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2022-01-06 18:46 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2022-01-06 18:46 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2022-01-06 18:46 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2022-01-06 18:46 /labs/slaba03/laba03_views_programmes.csv


В laba03_train.csv содержатся факты покупки (колонка purchase) пользователями (колонка user_id) телепередач (колонка item_id). Такой формат файла вам уже знаком.

`laba03_items.csv` — дополнительные данные по items. В данном файле много лишней или ненужной информации, так что задача её фильтрации и отбора ложится на вас. Поля в файле, на которых хотелось бы остановиться:

- `item_id` — primary key. Соответствует item_id в предыдущем файле.
- `content_type` — тип телепередачи (1 — платная, 0 — бесплатная). Вас интересуют платные передачи.
- `title` — название передачи, текстовое поле.
- `year` — год выпуска передачи, число.
- `genres` — поле с жанрами передачи, разделёнными через запятую.

`laba03_test.csv` — тестовый датасет без указанного целевого признака purchase, который вам и предстоит предсказать.

Дополнительный файл `laba03_views_programmes.csv` по просмотрам передач с полями:

- `ts_start` — время начала просмотра.
- `ts_end` — время окончания просмотра.
- `item_type` — тип просматриваемого контента:
- `live` — просмотр "вживую", в момент показа контента в эфире.
- `pvr` — просмотр в записи, после показа контента в эфире.

#### Результат
Предсказание целевой переменной "купит/не купит" — хорошо знакомая вам задача бинарной классификации. Поскольку нам важны именно вероятности отнесения пары (пользователь, товар) к классу "купит" (1), то, на самом деле, вы можете подойти к проблеме с разных сторон:

Как просто к задаче бинарной классификации. У вас есть два датасета, которые можно каким-то образом объединить, дополнительно обработать и сделать предсказания классификаторами (Spark ML).
Как к разработке рекомендательной системы: рекомендовать пользователю user_id топ-N лучших телепередач, которые были найдены по методике user-user / item-item коллаборативной фильтрации.
Как к задаче факторизации матриц: алгоритмы SVD, ALS, FM/FFM.

#### Советы
На качество прогноза в большей степени влияет качество признаков, которые вы сможете придумать из имеющихся данных, нежели выбор и сложность алгоритма.
Качество входных данных также имеет сильное значение. Существует фраза "garbage in – garbage out". Мусор на входе – мусор на выходе. Потратьте время на подготовку и предобработку данных. Путь к успеху в третьей лабораторной:
Сосредоточьтесь на формировании следующих фичей: по файлу laba03_train.csv сформируйте признаки, характеризирующие как интенсивно покупает пользователь и "покупаемость" item'ов
возьмите достаточно мощную модель (например GBTClassifier из pyspark'а)

In [4]:
from pyspark.ml.feature import Tokenizer, OneHotEncoderEstimator, StringIndexer, VectorAssembler, CountVectorizer
from pyspark.ml import Pipeline

#### get train and test ids

In [5]:
test = ( 
    spark.read
    .option('header', 'True')
    .option('sep', ',')
    .csv('/labs/slaba03/laba03_test.csv')
)

In [6]:
test.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
+-------+-------+--------+
only showing top 5 rows



In [7]:
train = ( 
    spark.read
    .option('header', 'True')
    .option('sep', ',')
    .csv('/labs/slaba03/laba03_train.csv')
)

In [8]:
train.show(5)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
+-------+-------+--------+
only showing top 5 rows



In [9]:
train.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- purchase: string (nullable = true)



In [10]:
train = train.withColumn("purchase", train.purchase.cast('int'))

In [11]:
windowSpecAgg  = Window.partitionBy('user_id')
users_activity = train.withColumn('user_activity', F.sum('purchase').over(windowSpecAgg)/F.count('purchase').over(windowSpecAgg))\
.select('user_id', 'user_activity')\
.dropDuplicates(['user_id'])

In [12]:
users_activity

DataFrame[user_id: string, user_activity: double]

In [13]:
users_activity.select('user_id').distinct().count()

1941

In [14]:
users_activity.count()

1941

In [15]:
train.select('user_id').distinct().count()

1941

In [16]:
windowSpecAgg  = Window.partitionBy('item_id')
items_intenstiy = train.withColumn(
    'items_intenstiy', F.sum('purchase').over(windowSpecAgg)/F.count('purchase').over(windowSpecAgg))\
.select('item_id', 'items_intenstiy')\
.dropDuplicates(['item_id'])

In [17]:
items_intenstiy.select('item_id').distinct().count()

3704

In [18]:
items_intenstiy.count()

3704

In [19]:
train.select('item_id').distinct().count()

3704

In [20]:
train0 = train.join(users_activity, how='left', on='user_id')

In [21]:
train1 = train0.join(items_intenstiy, how='left', on='item_id')

In [22]:
train1.count()

5032624

In [23]:
train.count()

5032624

In [24]:
train1.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- user_activity: double (nullable = true)
 |-- items_intenstiy: double (nullable = true)



In [25]:
# do the same for test dataset

In [26]:
# items_freq = test.groupby('item_id').count().withColumnRenamed('count', 'items_cnt')
# users_activity = test.groupby('user_id').count().withColumnRenamed('count', 'users_cnt')

# test_new = test.join(items_freq, how='left', on='item_id')
# test_new = test_new.join(users_activity, how='left', on='user_id')

In [27]:
# test_new.printSchema()

#### get and preprocess data on items

In [28]:
items = ( 
    spark.read
    .option('header', 'True')
    .option('sep', '\t')
    .csv('/labs/slaba03/laba03_items.csv')
)

In [29]:
items = items.select('item_id', 'content_type', 'title', 'year', 'genres')

In [30]:
items.show(1, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------------
 item_id      | 65667                                         
 content_type | 1                                             
 title        | на пробах только девушки (all girl auditions) 
 year         | 2013.0                                        
 genres       | Эротика                                       
only showing top 1 row



In [31]:
items.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genres: string (nullable = true)



In [32]:
items.select('content_type').groupby('content_type').count().show()

+------------+------+
|content_type| count|
+------------+------+
|           0|631864|
|           1|  3704|
+------------+------+



In [33]:
# further work with only paid items
items = items.filter(F.col('content_type')==1)

In [34]:
# check that items are exhausive to train and test
test.select('item_id').join(items, how='left_anti', on='item_id').show()

+-------+
|item_id|
+-------+
+-------+



In [35]:
train.select('item_id').join(items, how='left_anti', on='item_id').show()

+-------+
|item_id|
+-------+
+-------+



In [36]:
items.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: string (nullable = true)
 |-- genres: string (nullable = true)



In [37]:
# items = items.fillna('', 'genres')
# tokenizer = Tokenizer(inputCol='genres', outputCol='genres_split')
# items = tokenizer.transform(items)

In [38]:
items = items.fillna('', 'genres')
genres_split = F.udf(lambda x: x.split(','), ArrayType(StringType()))
items = items.withColumn('genres', genres_split(F.col('genres')))


In [39]:
cv = CountVectorizer(inputCol="genres", outputCol="genres_features")
model = cv.fit(items)
items = model.transform(items).drop('genres')

In [40]:
items

DataFrame[item_id: string, content_type: string, title: string, year: string, genres_features: vector]

In [41]:
train2 = train1.join(items.select('item_id', 'genres_features'), how='left', on='item_id')

In [42]:
train2

DataFrame[item_id: string, user_id: string, purchase: int, user_activity: double, items_intenstiy: double, genres_features: vector]

In [43]:
# check for nan colums
train2_cols=["item_id","user_id", "purchase", "user_activity", "items_intenstiy"]
train2.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in train2_cols]
   ).show()

+-------+-------+--------+-------------+---------------+
|item_id|user_id|purchase|user_activity|items_intenstiy|
+-------+-------+--------+-------------+---------------+
|      0|      0|       0|            0|              0|
+-------+-------+--------+-------------+---------------+



#### prep for model

In [44]:
assembler = (
    VectorAssembler()
    .setInputCols(['user_activity', 'items_intenstiy', 'genres_features']) # 'year',
    .setOutputCol('features')
)

In [45]:
train_for_model = assembler.transform(train2)

In [46]:
train_df, test_df = train_for_model.randomSplit([0.75, 0.25], seed = 123)

In [47]:
from pyspark.ml.classification import LogisticRegression

In [48]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'purchase', maxIter=5)

In [49]:
lrModel = lr.fit(train_df)

In [50]:
predictions = lrModel.transform(test_df)

In [51]:
predictions

DataFrame[item_id: string, user_id: string, purchase: int, user_activity: double, items_intenstiy: double, genres_features: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [52]:
predictions.select('user_id','item_id','probability').show(5)

+-------+-------+--------------------+
|user_id|item_id|         probability|
+-------+-------+--------------------+
| 510087| 100140|[0.99821724412245...|
| 517612| 100140|[0.99832471089242...|
| 520446| 100140|[0.99817804077684...|
| 566758| 100140|[0.99832542454817...|
| 613775| 100140|[0.99832475100011...|
+-------+-------+--------------------+
only showing top 5 rows



In [53]:
get_element = F.udf(lambda v:float(v[1]), FloatType())
predictions.select('user_id', 'item_id', get_element('probability').alias('purchase')).show(5)

+-------+-------+------------+
|user_id|item_id|    purchase|
+-------+-------+------------+
| 510087| 100140|0.0017827558|
| 517612| 100140|0.0016752891|
| 520446| 100140|0.0018219593|
| 566758| 100140|0.0016745755|
| 613775| 100140| 0.001675249|
+-------+-------+------------+
only showing top 5 rows



In [54]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
evaluator = BinaryClassificationEvaluator(labelCol = 'purchase')
print('ROC AUC Train', evaluator.evaluate(predictions))

ROC AUC Train 0.8731953545669617


In [56]:
# fit model on the whole dataset

In [57]:
lrModel = lr.fit(train_for_model)

In [58]:
# prepare test dataset

In [59]:
test

DataFrame[user_id: string, item_id: string, purchase: string]

In [60]:
user_activity_df = train.groupBy('user_id').agg(
    (F.sum('purchase')/F.count('purchase')
    ).alias('user_activity'))

item_intensivity_df = train.groupBy('item_id').agg(
    (F.sum('purchase')/F.count('purchase')
    ).alias('item_intensivity'))


test = test.join(user_activity_df, how='left', on='user_id')
test = test.join(item_intensivity_df, how='left', on='item_id')
test = test.join(items.select('item_id', 'genres_features'), how='left', on='item_id')

In [61]:
test.show(5)

+-------+-------+--------+--------------------+----------------+---------------+
|item_id|user_id|purchase|       user_activity|item_intensivity|genres_features|
+-------+-------+--------+--------------------+----------------+---------------+
| 100140| 763464|    null|3.933910306845004E-4|             0.0| (84,[2],[1.0])|
| 100140| 775479|    null|0.001520912547528517|             0.0| (84,[2],[1.0])|
| 100140| 887290|    null|                 0.0|             0.0| (84,[2],[1.0])|
| 100140| 895407|    null|0.003097173828881146|             0.0| (84,[2],[1.0])|
| 100140| 896973|    null|7.727975270479134E-4|             0.0| (84,[2],[1.0])|
+-------+-------+--------+--------------------+----------------+---------------+
only showing top 5 rows



In [62]:
assembler_test = (
    VectorAssembler()
    .setInputCols(['user_activity', 'item_intensivity', 'genres_features']) 
    .setOutputCol('features')
)

test_model = assembler_test.transform(test)

In [63]:
test_model.show(4)

+-------+-------+--------+--------------------+----------------+---------------+--------------------+
|item_id|user_id|purchase|       user_activity|item_intensivity|genres_features|            features|
+-------+-------+--------+--------------------+----------------+---------------+--------------------+
| 100140| 748042|    null|0.001530807500956...|             0.0| (84,[2],[1.0])|(86,[0,4],[0.0015...|
| 100140| 855465|    null|0.001552795031055...|             0.0| (84,[2],[1.0])|(86,[0,4],[0.0015...|
| 100140| 905618|    null|                 0.0|             0.0| (84,[2],[1.0])|      (86,[4],[1.0])|
| 100140| 871154|    null|                 0.0|             0.0| (84,[2],[1.0])|      (86,[4],[1.0])|
+-------+-------+--------+--------------------+----------------+---------------+--------------------+
only showing top 4 rows



In [64]:
#### log regression

In [65]:
log_test_predictions = lrModel.transform(test_model)

In [66]:
log_test_predictions

DataFrame[item_id: string, user_id: string, purchase: string, user_activity: double, item_intensivity: double, genres_features: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double]

Важно! Для точной проверки не забудьте отсортировать полученный файл по возрастанию идентификаторов пользователей (user_id), а затем — по возрастанию идентификаторов передач (item_id).

In [67]:
test_predictions_pd = log_test_predictions.select('user_id', 'item_id', get_element('probability').alias('purchase')) \
.orderBy('user_id', 'item_id') \
.toPandas()

In [ ]:
spark.stop()